In [ ]:
    """
    Load current state from file if file does not exist, 
    initialize it with default values using initialize_state()
    return: dict - The current state dictionary
    """
    try:
        if SHARED_STATE_FILE.exists():
            with open(SHARED_STATE_FILE, 'r') as f:
                # read file and change current view to SUMMARY
                state = json.load(f)
            state["current_view"] = "SUMMARY"
            #save current view change
            with open(SHARED_STATE_FILE, 'w') as f:
                json.dump(state, f)
            return state
        else:
            return initialize_state()
    except:
        return initialize_state()

In [ ]:
def chhose_tranactions_view(account, console):
    """
    Choose to view all transactions or filter by amount
    param account: dict - The account dictionary
    param console: Console - The Rich Console object to use for output
    """
    choice = Prompt.ask(
        "View Transactions: [1] All Transactions, [2] Filter by Amount , [3] Filter by Account",
        choices=["1", "2", "3"]
    )

    transactions = get_transactions(account)

    if choice == "1":
        return transactions
    elif choice == "2":
        amount_threshold = float(Prompt.ask("Enter amount threshold: "))
        filtered_transactions = [trans for trans in transactions if trans['amount'] >= amount_threshold]
        return filtered_transactions
    elif choice == "3":
        account_name = Prompt.ask("Enter Account Name: ")
        filtered_transactions = [trans for trans in transactions if trans['account_name'] == account_name]
        return filtered_transactions


In [ ]:
def view_transactions_for_account(state, console):
    """
    View transactions for a specific account
    param state: dict - The current state dictionary
    param console: Console - The Rich Console object to use for output
    """
    accounts = state.get("accounts", [])
    account = choose_account(accounts, console)
    # save account to current state as current_account_view
    state["current_account_view"] = account['name']
    write_shared_state(state)


    if account:
        transactions = get_transactions(account)
        console.print(f"\n[yellow]Transactions for account '{account['name']}':[/yellow]")
        if transactions:
            for idx, transaction in enumerate(transactions, start=1):
                console.print(f"{idx}. Amount: {transaction['amount']}, Description: {transaction['description']}")
        else:
            console.print("[red]No transactions available for this account.[/red]")
    else:
        console.print("[red]Account not found.[/red]")